In [43]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/measures/measurement_data.xlsx
/kaggle/input/measures/measurement_test.xlsx


In [44]:
from sklearn.model_selection import train_test_split
# Load the dataset
df_train = pd.read_excel("/kaggle/input/measures/measurement_data.xlsx")
df_test = pd.read_excel("/kaggle/input/measures/measurement_test.xlsx")

print("loaded")
print(df_train.shape)
print(df_test.shape)

loaded
(2651, 7)
(662, 7)


In [45]:
x_train=df_train[df_train.columns[:3]]
y_train=df_train.iloc[:, 3:6]
x_test=df_test[df_test.columns[:3]]

# Fill missing values in x_train, y_train, and x_test with the column mean
x_train.fillna(x_train.mean(), inplace=True)

# Convert y_train to numeric, forcing non-numeric values to NaN
y_train = y_train.apply(pd.to_numeric, errors='coerce')
# Fill missing values in y_train with the column mean
y_train.fillna(y_train.mean(), inplace=True)

x_test = x_test.apply(pd.to_numeric, errors='coerce')
x_test.fillna(x_test.mean(), inplace=True)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(2651, 3)
(2651, 3)
(662, 3)


/tmp/ipykernel_30/2656591952.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train.fillna(x_train.mean(), inplace=True)


In [46]:
# Split the data into training and validation sets
x_train_split, x_valid_split, y_train_split, y_valid_split = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
print('done')

done


In [47]:
from sklearn.linear_model import LinearRegression

# Initialize the Linear Regression model
model = LinearRegression()

# Train the model
model.fit(x_train_split, y_train_split)
print('fitted')

fitted


In [48]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.metrics import mean_absolute_error

pred=model.predict(x_valid_split)

# Calculate MAPE for each target variable
mape_per_target = np.mean(np.abs((y_valid_split - pred) / y_valid_split), axis=0) * 100
# Display MAPE for each target variable
print("MAPE for each target:", mape_per_target)
# Calculate variance for each target in the validation set to weight MAPE
target_variances = np.var(y_valid_split, axis=0)
# Calculate weighted MAPE across all targets
weighted_mape = np.average(mape_per_target, weights=target_variances)
print("Weighted MAPE (overall performance):", weighted_mape, "%")

# rmse=mean_squared_error(pred, y_valid_split)
# print("root mean squared error:", rmse)
# print("R-squared:", r2_score(y_valid_split, pred))


MAPE for each target: Length (L-M1)    3.007403
Drape (D-M3)     3.412858
Pallu (P-M4)     0.819683
dtype: float64
Weighted MAPE (overall performance): 3.351551995949391 %


In [49]:
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error

knn_regressor = KNeighborsRegressor(n_neighbors=7)
knn_regressor.fit(x_train_split, y_train_split)
y_pred_regress = knn_regressor.predict(x_valid_split)

# Calculate MAPE for each target variable
mape_per_target = np.mean(np.abs((y_valid_split - y_pred_regress) / y_valid_split), axis=0) * 100
# Display MAPE for each target variable
print("MAPE for each target, length, drape, pallu:", mape_per_target)
# Calculate variance for each target in the validation set to weight MAPE
target_variances = np.var(y_valid_split, axis=0)
# Calculate weighted MAPE across all targets
weighted_mape = np.average(mape_per_target, weights=target_variances)

print("Weighted MAPE (overall model performance):", weighted_mape, "%")

#print("Root Mean Squared Error:", np.sqrt(mean_squared_error(y_valid_split, y_pred_regress)))
#print("R-squared:", r2_score(y_valid_split, y_pred_regress))

MAPE for each target, length, drape, pallu: Length (L-M1)    2.401895
Drape (D-M3)     1.794276
Pallu (P-M4)     0.667393
dtype: float64
Weighted MAPE (overall model performance): 1.8479028767272434 %


In [50]:
# After training the Linear Regression and KNN models
def get_user_input():
    """
    Function to prompt for user input of three feature values.
    """
    print("Please enter values for the following features:")
    feature1 = float(input("Enter the value of waist measure: "))
    feature2 = float(input("Enter the value of bust measure: "))
    feature3 = float(input("Enter the value of height (in inches only): "))
    user_data = pd.DataFrame([[feature1, feature2, feature3]], columns=x_train.columns)
    return user_data
    print(user_data)



In [51]:
# Get user input for features
user_input = get_user_input()

# Make predictions using the trained models
lin_reg_prediction = model.predict(user_input)
knn_prediction = knn_regressor.predict(user_input)

# Display the predictions
print("\nPredictions for the input values:")
print(f"Linear Regression Prediction: {lin_reg_prediction[0]}")
print(f"KNN Regression Prediction: {knn_prediction[0]}")

Please enter values for the following features:


Enter the value of waist measure:  34
Enter the value of bust measure:  36
Enter the value of height (in inches only):  60



Predictions for the input values:
Linear Regression Prediction: [40.08001792 51.56143569 43.4343122 ]
KNN Regression Prediction: [40.57142857 53.14285714 43.28571429]
